In [30]:
import pandas as pd
import numpy as np
import os
import json
import statistics
import pandas as pd
import numpy as np
import scipy.stats as stats
import researchpy as rp
import statsmodels.api as sm
from statsmodels.formula.api import ols
    
import matplotlib.pyplot as plt

In [2]:
d = "./result"

dirs_to_analyze = [os.path.join(d, o) for o in os.listdir(d) 
                    if os.path.isdir(os.path.join(d,o)) and "\\experiment" in os.path.join(d, o)]


In [53]:
rowsList = []
for curdir in dirs_to_analyze:
    row = {}
    try:
        config = json.load(open(curdir + "/config.json"))
        row['Executor Memory'] = int(config['jobClassParameters'][0]['systemParameters']['executorMemory'][:1])
        row['Batch Size'] = int(config['jobClassParameters'][0]['hyperParameters']['batchSize'])
        row['Max Epochs'] = int(config['jobClassParameters'][0]['hyperParameters']['maxEpoch'])
        row['Learning Rate'] = float(config['jobClassParameters'][0]['hyperParameters']['learningRate'])
        row['Learning Rate Decay'] = float(config['jobClassParameters'][0]['hyperParameters']['learningrateDecay'])
        row['Nodes'] = int(config['numberOfNodes'])
    except Exception as e:
        print("Error parsing config for folder " + curdir)
        print(e)
        
    try:
        accuracy = pd.read_json(curdir + "/accuracy.json").transpose()
        row['Top 1 mean'] = accuracy["top1"].mean()
        row['Top 1 std'] = accuracy["top1"].std()
        row['Top 5 mean'] = accuracy["top5"].mean()
        row['Top 5 std'] = accuracy["top5"].std()
        row['loss mean'] = accuracy["loss"].mean()
        row['loss std'] = accuracy["loss"].std()
        row['lossnum'] = accuracy["lossnum"].mean()
    except Exception as e:
        print("Error parsing accuracy for folder " + curdir)
        print(e)
        
        
    try:
        time_summary = json.load(open(curdir + "/summary.json"))
        service_times = []
        work_times = []
        process_times = []
        
        for job in time_summary['jobs']:
            service_time = float(job['end']) - float(job['arrive'])
            service_times.append(service_time)
            wait_time = float(job['start']) - float(job['arrive'])
            wait_times.append(wait_time)
            process_time = float(job['end']) - float(job['start'])
            process_times.append(process_time)

        row['Service Time mean'] = statistics.mean(service_times)
        row['Service Time std'] = statistics.stdev(service_times)
        row['Process Time mean'] = statistics.mean(process_times)
        row['Process Time std'] = statistics.stdev(process_times)
        row['Waiting Time mean'] = statistics.mean(wait_times)
        row['Waiting Time std'] = statistics.stdev(wait_times)
    except Exception as e:
        print("Error parsing time summary for folder " + curdir)
        print(e)
        
    
    rowsList.append(row) 
    
analysis = pd.DataFrame(rowsList)
analysis = analysis.drop([25, 21, 10])
analysis

Error parsing accuracy for folder ./result\experiment1_0.01_32_10_2G_2_0.01
Expected object or value
Error parsing time summary for folder ./result\experiment1_0.01_32_10_2G_2_0.01
[Errno 2] No such file or directory: './result\\experiment1_0.01_32_10_2G_2_0.01/summary.json'


,Executor Memory,Batch Size,Max Epochs,Learning Rate,Learning Rate Decay,Nodes,Top 1 mean,Top 1 std,Top 5 mean,Top 5 std,loss mean,loss std,lossnum,Service Time mean,Service Time std,Process Time mean,Process Time std,Waiting Time mean,Waiting Time std
0,1,128,10,0.001,0.001,1,0.638575,0.040876,0.946263,0.014880,1.907845,0.108069,157.0,1776.609323,989.718032,443.521094,23.668845,948.972820,824.493813
1,1,128,10,0.001,0.001,2,0.606055,0.046856,0.939273,0.019292,1.969239,0.058131,157.0,2120.792949,645.245517,656.160590,330.464160,951.016655,825.241355
2,2,128,10,0.001,0.001,1,0.648443,0.021566,0.957329,0.006531,1.913965,0.066078,157.0,1766.907113,935.187775,475.978486,11.339233,951.957123,825.569726
3,1,128,5,0.001,0.001,1,0.528279,0.065775,0.906986,0.028010,2.105443,0.046931,157.0,1670.351884,860.363370,268.185179,6.783555,954.434691,826.282179
4,1,128,5,0.001,0.001,2,0.573604,0.051022,0.920057,0.024303,2.077709,0.060236,157.0,1321.995722,393.015358,309.784817,80.674195,954.930044,823.762996
5,2,128,5,0.001,0.001,1,0.549371,0.074137,0.915879,0.035446,2.082460,0.049013,157.0,1668.154473,862.796170,268.689102,6.599487,957.342252,824.464941
6,2,128,5,0.001,0.001,2,0.556050,0.040288,0.920758,0.018385,2.100687,0.046055,157.0,1161.089359,375.572796,290.346777,36.715810,956.544098,821.542098
7,1,32,10,0.001,0.001,1,0.724250,0.009970,0.975650,0.000778,1.289145,0.002442,625.0,1954.859985,853.568007,1340.711881,14.968273,956.281323,821.457702
8,1,32,10,0.001,0.001,2,0.733275,0.020606,0.977575,0.000830,1.329912,0.099123,625.0,2075.957269,768.054586,1405.175199,44.450360,955.843776,821.323936
9,2,32,10,0.001,0.001,1,0.718750,0.016476,0.977800,0.001414,1.338503,0.138019,625.0,1959.242099,844.162402,1339.577599,32.175336,955.586365,821.241087


In [54]:
# Process time results:
print("Max avg process time: ", analysis["Process Time mean"].max())
print("Min avg process time: ", analysis["Process Time mean"].min())
print("Mean avg process time: ", analysis["Process Time mean"].mean())

print("Min mu: ", "%.7f" % (1/analysis["Process Time mean"].max()))
print("Max mu: ", "%.7f" % (1/analysis["Process Time mean"].min()))

Max avg process time:  1603.637364089489
Min avg process time:  266.5028019632612
Mean avg process time:  711.4513971013635
Min mu:  0.0006236
Max mu:  0.0037523


In [55]:
important_cols = [x for x in analysis.columns if "Time" in x]

In [56]:
# Executor Memory
from IPython.display import HTML, display

to_show = analysis.columns[0:5]
for col in to_show:
    display(rp.summary_cont(analysis[important_cols].groupby(analysis[col])))

Service Time mean                                      \
                                N         Mean          SD         SE   
Executor Memory                                                         
1                              16  1858.418670  330.387697  82.596924   
2                              11  1796.221435  255.870808  77.147951   

                                          Service Time std              \
                   95% Conf.     Interval                N        Mean   
Executor Memory                                                          
1                1696.528699  2020.308642               16  819.037702   
2                1645.011450  1947.431420               11  852.257593   

                                        ... Waiting Time mean            \
                         SD         SE  ...                SD        SE   
Executor Memory                         ...                               
1                204.995425  51.248856  ...          4.187306  1.046826   
2                175.390676  52.882278  ...          4.498926  1.356477   

                                        Waiting Time std              \
                  95% Conf.    Interval                N        Mean   
Executor Memory                                                        
1                947.751994  951.855553               16  823.783475   
2                948.079170  953.396561               11  823.623657   

                                                             
                       SD        SE   95% Conf.    Interval  
Executor Memory                                              
1                1.478170  0.369543  823.059172  824.507779  
2                1.381425  0.416515  822.807287  824.440027  

[2 rows x 36 columns]

Service Time mean                                                   \
                           N         Mean          SD         SE    95% Conf.   
Batch Size                                                                      
32                        14  2017.866717  138.082431  36.904082  1945.534716   
128                       13  1634.076960  299.515388  83.070622  1471.258540   

                        Service Time std                                    \
               Interval                N        Mean          SD        SE   
Batch Size                                                                   
32          2090.198717               14  899.062718  104.528138  27.93632   
128         1796.895380               13  760.966054  237.095104  65.75835   

            ... Waiting Time mean                                    \
            ...                SD        SE   95% Conf.    Interval   
Batch Size  ...                                                       
32          ...          4.360618  1.165424  945.757294  950.325757   
128         ...          2.744528  0.761195  951.000022  953.983906   

           Waiting Time std                                              \
                          N        Mean        SD        SE   95% Conf.   
Batch Size                                                                
32                       14  822.897305  0.969446  0.259095  822.389478   
128                      13  824.602582  1.297597  0.359889  823.897200   

                        
              Interval  
Batch Size              
32          823.405132  
128         825.307963  

[2 rows x 36 columns]

Service Time mean                                                   \
                           N         Mean          SD         SE    95% Conf.   
Max Epochs                                                                      
5                         15  1733.147148  331.294563  85.539888  1565.488967   
10                        12  1957.993941  200.823365  57.972712  1844.367426   

                        Service Time std                                     \
               Interval                N        Mean          SD         SE   
Max Epochs                                                                    
5           1900.805329               15  814.515782  239.836228  61.925448   
10          2071.620457               12  855.141668  108.232252  31.243960   

            ... Waiting Time mean                                    \
            ...                SD        SE   95% Conf.    Interval   
Max Epochs  ...                                                       
5           ...          4.487906  1.158772  947.795330  952.337718   
10          ...          4.141226  1.195469  947.988467  952.674705   

           Waiting Time std                                              \
                          N        Mean        SD        SE   95% Conf.   
Max Epochs                                                                
5                        15  823.841002  1.134907  0.293032  823.266660   
10                       12  823.565066  1.744729  0.503660  822.577893   

                        
              Interval  
Max Epochs              
5           824.415345  
10          824.552240  

[2 rows x 36 columns]

Service Time mean                                      \
                              N         Mean          SD         SE   
Learning Rate                                                         
0.001                        14  1808.078128  280.120813  74.865436   
0.010                        13  1860.003133  326.471271  90.546839   

                                        Service Time std              \
                 95% Conf.     Interval                N        Mean   
Learning Rate                                                          
0.001          1661.341872  1954.814383               14  813.447443   
0.010          1682.531328  2037.474937               13  853.167119   

                                      ... Waiting Time mean            \
                       SD         SE  ...                SD        SE   
Learning Rate                         ...                               
0.001          209.720597  56.050187  ...          4.603124  1.230237   
0.010          173.865206  48.221532  ...          3.071921  0.851997   

                                      Waiting Time std                        \
                95% Conf.    Interval                N        Mean        SD   
Learning Rate                                                                  
0.001          949.456472  954.278999               14  823.572468  1.646084   
0.010          946.701515  950.041346               13  823.875483  1.161215   

                                                 
                     SE   95% Conf.    Interval  
Learning Rate                                    
0.001          0.439934  822.710196  824.434739  
0.010          0.322063  823.244240  824.506727  

[2 rows x 36 columns]

Service Time mean                                      \
                                    N         Mean          SD         SE   
Learning Rate Decay                                                         
0.001                              14  1808.078128  280.120813  74.865436   
0.010                              13  1860.003133  326.471271  90.546839   

                                              Service Time std              \
                       95% Conf.     Interval                N        Mean   
Learning Rate Decay                                                          
0.001                1661.341872  1954.814383               14  813.447443   
0.010                1682.531328  2037.474937               13  853.167119   

                                            ... Waiting Time mean            \
                             SD         SE  ...                SD        SE   
Learning Rate Decay                         ...                               
0.001                209.720597  56.050187  ...          4.603124  1.230237   
0.010                173.865206  48.221532  ...          3.071921  0.851997   

                                            Waiting Time std              \
                      95% Conf.    Interval                N        Mean   
Learning Rate Decay                                                        
0.001                949.456472  954.278999               14  823.572468   
0.010                946.701515  950.041346               13  823.875483   

                                                                 
                           SD        SE   95% Conf.    Interval  
Learning Rate Decay                                              
0.001                1.646084  0.439934  822.710196  824.434739  
0.010                1.161215  0.322063  823.244240  824.506727  

[2 rows x 36 columns]